# Práctica Python
## Alumnos: _Tania Batista, Fernando de la Fuente, Lourdes Ruiz_
## Tema: Análisis del S&P 500
### Profesor: _Miguel Angel Sicilia_

Hemos elegido para la práctica una tabla de datos que contiene las cotizaciones de 503 valores que componen el índice de bolsa americana S&P 500 de 5 años, desde el 13 de Agosto de 2012 hasta el 11 de Agosto de 2017.
Los datos están extraídos de Kaggle, en el siguiente repositorio:

[Kaggle S&P 500](https://www.kaggle.com/camnugent/sandp500/)

A partir de estos datos, sin tener los datos del índice, sino los de los valores que lo componen, haremos algunos estudios, así como la composición de un índice sintético.
Deicidimos componer un sintético debido a que:
* El índice real varía diariamente el peso de cada uno de los valores que lo componen, y el acceso a esta información que es necesaria para componer el índice real es de pago. 
* Periodicamente se reune un comité que decide qué valores entran y salen del índice, y a lo largo de los 5 años de los que tenemos datos ha habido varios valores que han entrado y otros que han salido. Nuestra base de datos contiene información de los 503 valores que componían el índice en fecha 14 de Agosto de 2017, y sobre esta información es sobre la que trabajaremos.

Por tanto, debido a estas limitaciones no es posible recomponer el índice real y comprobar los datos obtenidos con los de cualquier otra fuente, y en consecuencia fabricaremos uno sintético asumiendo que en el periodo no ha variado la composición, y todos los valores tienen el mismo peso.

Comenzamos por tanto la práctica.

Lo primero sería cargar los datos, y volcarlos a un data frame.

In [ ]:
# PRUEBA PARA ABRIR EL FICHERO DESDE LA URL, ESTA COMPRIMIDO EN ORIGEN
import pandas as pd
import bz2
path ='https://www.kaggle.com/camnugent/sandp500/downloads/all_stocks_5yr.csv.bz2'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0)
sp = pd.DataFrame(sp500)
print(sp.head(10))

In [ ]:
# PRUEBA PARA ABRIR EL FICHERO DESDE LA CARPETA DESCARGAS, ESTA COMPRIMIDO AL DESCARGARLO. --> FUNCIONA
import pandas as pd
path ='C:\\Users\\ffuen\\Downloads\\all_stocks_5yr.csv.zip'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0)
print(sp500.head(10))

In [71]:
import pandas as pd
import numpy as np
#import csv, operator
path ='C:\\Users\\ffuen\\Desktop\\all_stocks_5yr.csv'
#path ='https://www.kaggle.com//camnugent//sandp500//downloads//all_stocks_5yr.csv'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0, )
sp = pd.DataFrame(sp500)
print(sp.head(10))
print(sp.tail(10))

         Date   Open   High    Low  Close     Volume Name
0  2012-08-13  92.29  92.59  91.74  92.40  2075391.0  MMM
1  2012-08-14  92.36  92.50  92.01  92.30  1843476.0  MMM
2  2012-08-15  92.00  92.74  91.94  92.54  1983395.0  MMM
3  2012-08-16  92.75  93.87  92.21  93.74  3395145.0  MMM
4  2012-08-17  93.93  94.30  93.59  94.24  3069513.0  MMM
5  2012-08-20  94.00  94.17  93.55  93.89  1640008.0  MMM
6  2012-08-21  93.98  94.10  92.99  93.21  2302988.0  MMM
7  2012-08-22  92.56  93.36  92.43  92.68  2463908.0  MMM
8  2012-08-23  92.65  92.68  91.79  91.98  1823757.0  MMM
9  2012-08-24  92.03  92.97  91.94  92.83  1945796.0  MMM
              Date   Open   High    Low  Close     Volume Name
606791  2017-07-31    NaN    NaN  62.32  62.52  2130374.0  ZTS
606792  2017-08-01  62.60  62.69  61.92  62.27  1750974.0  ZTS
606793  2017-08-02  62.04  62.21  61.38  61.54  1831228.0  ZTS
606794  2017-08-03  61.23  62.47  61.20  62.28  1768815.0  ZTS
606795  2017-08-04  62.37  62.85  61.86  62.12 

Comprobamos la cantidad de información que tiene la tabla, en filas y columnas:

In [78]:
print(sp.shape)
print("El número de dimensiones del data frame es de",sp.ndim)
print("El número de filas del data frame es de",len(sp.index))
print(sp.index)
print("El número de columnas del data frame es de",len(sp.columns))
print(sp.columns)
print(type(sp500))

(606801, 7)
El número de dimensiones del data frame es de 2
El número de filas del data frame es de 606801
RangeIndex(start=0, stop=606801, step=1)
El número de columnas del data frame es de 7
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name'], dtype='object')
<class 'pandas.core.frame.DataFrame'>


In [81]:
# Eliminar esta celda, tarda mucho en ejecutar, la siguiente hace lo mismo más rápido.
#Create a new function:
def nan_numero(x):
  return sum(x.isnull())

#Applying per column:
print ("Missing values per column:")
print (sp.apply(nan_numero, axis=0)) #axis=0 defines that function is to be applied on each column

Missing values per column:
Date        0
Open      384
High      208
Low       227
Close       0
Volume    406
Name        0
dtype: int64


Como hemos visto anteriormente, la tabla hacia el final tiene algunos campos vacíos, que se suplen con 'NaN'. 

Es necesario identificarlos, y sustituirlos para limpiar la tabla antes de trabajar con ella:

In [82]:
print(sp.isnull().any())
print("/nEl número de campos vacíos en la columna Date es",sp["Date"].isnull().sum())
print("El número de campos vacíos en la columna Open es",sp["Open"].isnull().sum())
print("El número de campos vacíos en la columna High es",sp["High"].isnull().sum())
print("El número de campos vacíos en la columna Low es",sp["Low"].isnull().sum())
print("El número de campos vacíos en la columna Close es",sp["Close"].isnull().sum())
print("El número de campos vacíos en la columna Volume es",sp["Volume"].isnull().sum())
print("El número de campos vacíos en la columna Name es",sp["Name"].isnull().sum())

Date      False
Open       True
High       True
Low        True
Close     False
Volume     True
Name      False
dtype: bool
/nEl número de campos vacíos en la columna Date es 0
El número de campos vacíos en la columna Open es 384
El número de campos vacíos en la columna High es 208
El número de campos vacíos en la columna Low es 227
El número de campos vacíos en la columna Close es 0
El número de campos vacíos en la columna Volume es 406
El número de campos vacíos en la columna Name es 0


Como vemos, los campos vacíos se encuentran en algunas columnas, no todas. 
Antes de decidir qué hacer con ellos, hagamos un análisis de qué información es la que contiene cada columna:
* _Date:_ Contiene la fecha de la información de cada fila.
* _Open:_ Es el primer precio al que abrió el valor ese día.
* _High:_ El precio más alto negociado en el día.
* _Low:_ El precio más bajo negociado en el día.
* _Close:_ Indica a qué precio cerró el valor ese día. Este podemos decir que es el dato más importante de la fila.
* _Volume:_ Indica el número de acciones negociadas en el día. No todas cambian de manos al mismo precio.
* _Name:_ Indica el nombre del valor. Es una abreviatura utilizada para identificar cada valor. Por ejemplo: Amazon sería AMZN.

In [7]:
print("La columna Name contiene",len(sp.Name.unique()),"valores distintos.")

print("Son las compañías que formaban el índice S&P en fecha 11 de Agosto de 2017.")

La columna Name contiene 503 valores distintos.
Son las compañías que formaban el índice S&P en fecha 11 de Agosto de 2017.


Para operar mejor y transformar los campos vacíos, crearemos una lista de cada columna del dataframe.

**ESTO ELIMINARLO, AL FINAL NO HA HECHO FALTA, ERA UNA PRUEBA.**

In [7]:
col_date = np.array(sp['Date'])
col_open = np.array(sp["Open"])
col_high = np.array(sp["High"])
col_low = np.array(sp["Low"])
col_close = np.array(sp["Close"])
col_volume = np.array(sp["Volume"])
col_name = np.array(sp["Name"])
columns_name = np.array(sp.columns)

Vamos a sustituir los NaN que aparezcan en las columnas High y Low por un valor, que será:
* Para la columna _High_ el resultado de incrementar el precio de cierre de ese día por la diferencia (en %) entre el High y el Close --> High = Close * ( 1 + (% diferencia media entre High y Close de todo el periodo)
* Para la columna _Low_ el resultado de disminuir el precio de cierre de ese día por la diferencia (en %) entre el Low y el Close --> Low = Close * ( 1 - (% diferencia media entre el Close y el Low de todo el periodo)

Para hacer esta variación de datos, vamos a calcular primero por cada valor el % que habrá que añadir o restar al Close en caso de que el valor sea NaN. Lo haremos con dos funciones, una para columna _High_ y otra para la columna _Low_.

In [72]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL HIGH, y SUSTITUIMOS LOS NAN POR ESTE CALCULO

def calculo_reemplazo_high(name):
    subset = sp[sp['Name'] == name]
    subset_high = subset['High']
    subset_close = subset['Close']
    media = np.mean( (subset_high - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio)
    return media

# Recorremos los campos que tienen algún NaN en la columna High (son 208), y los sustituimos.

print("El número de campos vacíos en la columna High antes de la sustitución es",sp['High'].isnull().sum())

for posiciones in sp['High'].index[sp['High'].apply(np.isnan)]:
    sp.loc[posiciones,'High'] = sp['Close'][posiciones] * (1 + calculo_reemplazo_high(sp['Name'][posiciones]))
    
print("El número de campos vacíos en la columna High después de la sustitución es",sp['High'].isnull().sum())

El número de campos vacíos en la columna High antes de la sustitución es 208
El número de campos vacíos en la columna High después de la sustitución es 0


In [73]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL LOW, y SUSTITUIMOS LOS NAN POR ESTE CALCULO

def calculo_reemplazo_low(name):
    subset = sp[sp['Name'] == name]
    subset_low = subset['Low']
    subset_close = subset['Close']
    media = np.mean( (subset_low - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio
    return media

# Recorremos los campos que tienen algún NaN en la columna Low (son 227), y los sustituimos.

print("El número de campos vacíos en la columna Low antes de la sustitución es",sp['Low'].isnull().sum())

for posiciones in sp['Low'].index[sp['Low'].apply(np.isnan)]:
    sp.loc[posiciones,'Low'] = sp['Close'][posiciones] * (1 + calculo_reemplazo_low(sp['Name'][posiciones]))
    
print("El número de campos vacíos en la columna Low después de la sustitución es",sp['Low'].isnull().sum())

El número de campos vacíos en la columna Low antes de la sustitución es 227
El número de campos vacíos en la columna Low después de la sustitución es 0


In [69]:
# ESTA CELDA YA SE PUEDE BORRAR, LO HEMOS SUSTITUIDO POR LO DE ARRIBA: CALCULO DE VALOR Y SUSTITUCION EN MISMA CELDA

#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL HIGH
lista_valores = np.unique(sp['Name'])
dicc_high = {}
valores_nan_high = np.array([],dtype=int)

# Calculamos la lista de valores que tienen algún NaN en la columna High (son 207).
for posiciones in np.where(np.isnan(sp['High'])):
    valores_nan_high = np.append(valores_nan_high, sp['Name'][posiciones])
valores_nan_high = np.unique(valores_nan_high)
    
# En un diccionario vamos a almacenor, de cada valor, la media entre el High y el cierre. 
# Asignaremos a cada NaN que haya en la columna High el resultado de añadir este % al cierre de ese día para cada valor.

def calculo_reemplazo_colhigh(valores_nan_high, dicc_high):
    for valor in valores_nan_high:
        subset = sp[sp['Name'] == valor]
        subset_high = subset['High']
        subset_close = subset['Close']
        media = np.mean( (subset_high - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio
        dicc_high[valor]= media
    return dicc_high

dicc_high = calculo_reemplazo_colhigh(valores_nan_high, dicc_high)
print(dicc_high)

{'A': 0.00845903171842999, 'ABBV': 0.010426357313012459, 'ABC': 0.0078466686525526, 'ABT': 0.007511082154156873, 'AEE': 0.007168040956056294, 'AET': 0.009442710455496982, 'AGN': 0.012119237750734601, 'AIZ': 0.007635816524199367, 'AJG': 0.006062787770327242, 'ALB': 0.010416468188525806, 'ALK': 0.012098286575717595, 'ALL': 0.006016080921169382, 'ALLE': 0.009141903131215316, 'AME': 0.0072924764657328715, 'ANDV': 0.01487508952113967, 'ANTM': 0.009137966319035101, 'AON': 0.006425122156503418, 'AOS': 0.009076847245479682, 'APA': 0.013646078866829687, 'APD': 0.0072102859596091026, 'ARNC': 0.013224833019874392, 'AVB': 0.007989886233633439, 'AYI': 0.010498183232418946, 'BA': 0.007516224319201832, 'BAC': 0.009813867906742397, 'BAX': 0.007103905194702262, 'BCR': 0.007004464171556368, 'BF.B': 0.007642828093781286, 'BHF': 0.019877653367512804, 'BHGE': 0.019793821059792518, 'BSX': 0.009849850000190624, 'BWA': 0.010916457344937165, 'BXP': 0.007690158424180308, 'CAH': 0.007741315622887705, 'CAT': 0.00

In [49]:
# ESTA CELDA YA SE PUEDE BORRAR, LO HEMOS SUSTITUIDO POR LO DE ARRIBA: CALCULO DE VALOR Y SUSTITUCION EN MISMA CELDA

#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL LOW
lista_valores = np.unique(sp['Name'])
dicc_low = {}
valores_nan_low = np.array([],dtype=int)

# Calculamos la lista de valores que tienen algún NaN en la columna Low (son 224).
for posiciones in np.where(np.isnan(sp['Low'])):
    valores_nan_low = np.append(valores_nan_low, sp['Name'][posiciones])
valores_nan_low = np.unique(valores_nan_low)
    
# En un diccionario vamos a almacenor, de cada valor, la media entre el Low y el cierre. 
# Asignaremos a cada NaN que haya en la columna Low el resultado de añadir este % al cierre de ese día para cada valor.

def calculo_reemplazo_collow(valores_nan_low, dicc_low):
    for valor in valores_nan_low:
        subset = sp[sp['Name'] == valor]
        subset_low = subset['Low']
        subset_close = subset['Close']
        media = np.mean( (subset_low - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio
        dicc_low[valor]= media
    return dicc_low

dicc_low = calculo_reemplazo_collow(valores_nan_low, dicc_low)
print (dicc_low)

{'AAP': -0.010095553663446034, 'ABC': -0.008515193250688686, 'ACN': -0.007836611646280611, 'ADM': -0.00922492920174992, 'AEE': -0.00810951970456362, 'AEP': -0.007600069458626079, 'AES': -0.01130338594071607, 'AFL': -0.007276337621347446, 'AIG': -0.008529684430634987, 'AIV': -0.009149900341481894, 'AIZ': -0.008391614604847827, 'ALL': -0.006459622927913027, 'AMG': -0.010480457862372743, 'AMP': -0.009391147476276103, 'ANDV': -0.01582931928617878, 'APC': -0.013535431899561662, 'APH': -0.007771618456195819, 'AVY': -0.008521894352334365, 'AWK': -0.007858983699593836, 'AXP': -0.007374051319139385, 'AZO': -0.008426072404664387, 'BA': -0.008405098438480023, 'BAC': -0.009891755334526017, 'BBT': -0.008142459283595846, 'BBY': -0.014331573057196446, 'BCR': -0.007848501511447264, 'BDX': -0.007385226307735715, 'BEN': -0.009119034223288516, 'BHF': -0.008471952122438585, 'BHGE': -0.012067206262383534, 'BK': -0.008508310924812289, 'BLK': -0.009017282660509359, 'BMY': -0.009404691738007306, 'BRK.B': -0.0

In [6]:
# ESTA CELDA YA SE PUEDE BORRAR, LO HEMOS SUSTITUIDO POR LO DE ARRIBA: CALCULO DE VALOR Y SUSTITUCION EN MISMA CELDA

# Sustituímos los NaN de High por un dato que es Close X (1+ media de High para cada empresa del SP500).

for i in sp['High'].index[sp['High'].apply(np.isnan)]:
    sp.loc[i,'High'] = sp['Close'][i] * (1 + dicc_high[sp['Name'][i]])
print("El número de campos vacíos en la columna Low después de la sustitución es",sp['High'].isnull().sum())

El número de campos vacíos en la columna Low después de la sustitución es 0


In [7]:
# ESTA CELDA YA SE PUEDE BORRAR, LO HEMOS SUSTITUIDO POR LO DE ARRIBA: CALCULO DE VALOR Y SUSTITUCION EN MISMA CELDA

# Sustituímos los NaN de Low por un dato que es Close X (1+ media de Low para cada empresa del SP500).

for j in sp['Low'].index[sp['Low'].apply(np.isnan)]:
    sp.loc[i,'Low'] = sp['Close'][j] * (1 + dicc_low[sp['Name'][j]])
print("El número de campos vacíos en la columna Low después de la sustitución es",sp['Low'].isnull().sum())

El número de campos vacíos en la columna Low después de la sustitución es 0


A continuación vamos a sustituir los NaN que haya en al columna _Open_, por un número aleatorio que esté entre el valor de la columna _High_ y _Low_ de esa sesión.

In [74]:
# SUSTITUCION DE NaN EN COLUMNA Open, con un aleatorio que esté en el rango entre el High y el Low

print("El número de campos vacíos en la columna Open antes de la sustitución es",sp['Open'].isnull().sum())

for posiciones in sp['Open'].index[sp['Open'].apply(np.isnan)]: 
    sp.loc[posiciones,'Open'] = np.random.uniform (sp['Low'][posiciones], sp['High'][posiciones])

print("El número de campos vacíos en la columna Open después de la sustitución es",sp['Open'].isnull().sum())

El número de campos vacíos en la columna Open antes de la sustitución es 384
El número de campos vacíos en la columna Open después de la sustitución es 0


In [13]:
# BORRAR ESTA CELDA, LA DE ARRIBA LO HACE SIN WARNING Y MAS RAPIDO
# SUSTITUCION DE NaN EN COLUMNA Open, con un aleatorio que esté en el rango entre el High y el Low

def openfun(opencol):
    posiciones_nan_Open = np.where(np.isnan(opencol)) # esto nos da una lista con el número de fila de los NaN
    for filas in posiciones_nan_Open:
        opencol.loc[filas] = np.random.uniform (sp['Low'][filas], sp['High'][filas])
    return opencol
print("El número de campos vacíos en la columna Open antes de la sustitución es",sp['Open'].isnull().sum())
sp['Open'] = openfun(sp['Open'])
print("El número de campos vacíos en la columna Open después de la sustitución es",sp['Open'].isnull().sum())

El número de campos vacíos en la columna Open antes de la sustitución es 0
El número de campos vacíos en la columna Open después de la sustitución es 0


C:\Users\ffuen\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Respecto a la columna _Volume_, para cada valor que tenga NaN (aunque solo es un único valor) calcularemos la media del volumen de títulos que se negocian diariamente, y sustituiremos el NaN por esta media.

In [75]:
print("El número de campos vacíos en toda la tabla en la columna Volume es",sp["Volume"].isnull().sum())

a = sp['Volume'].index[sp['Volume'].apply(np.isnan)]
a = np.unique(sp.loc[a[0:-1],'Name'])
print("Los valores que tienen algún NaN en la columna Volume son: ",a)

for valores in a:
    sp[sp['Name']==valores] = sp[sp['Name']== valores].fillna(sp[sp['Name']== valores].mean())

print("El número de campos vacíos en la columna Volume es",sp["Volume"].isnull().sum())

El número de campos vacíos en toda la tabla en la columna Volume es 406
Los valores que tienen algún NaN en la columna Volume son:  ['GOOG']
El número de campos vacíos en la columna Volume es 0


Una vez hechos los reemplazos de los datos nulos en las columnas, comprobamos que no quede ninguno:

In [76]:
print(sp.isnull().any())

Date      False
Open      False
High      False
Low       False
Close     False
Volume    False
Name      False
dtype: bool


A partir de la información que tenemos ahora, datos diarios de 503 valores que componen el índice S&P 500, vamos a componer un índice nosotros, similar al original, para poder centrar parte de nuestro análisis en él.

### CONSIDERACIONES PRELIMINARES SOBRE EL ÍNDICE S&P 500
El índice S&P 500 es el índice más representativo de la evolución del mercado norteamericano, y recoge la evolución de las 500 mayores empresas cotizadas en los distintos mercados de renta variable del país. Hoy en día está formado por 503 valores, dado que algunos de ellos tienen distintas clases de acciones con diferentes derechos cotizando.
Un comité es el encargado de la selección de valores que lo componen, siguiendo 8 criterios, que son: capitalización bursatil, liquidez, domicilio, capital flotante, clasificación del sector, viabilidad financiera, periodo de tiempo durante el cual ha cotizado en bolsa y ser negociada en la bolsa de valores.

El comité se reune periódicamente y decide los valores que entran y salen del índice. Respecto a la ponderación de cada valor dentro del índice, esta depende de la capitalización de mercado de cada valor (precio diario X número de acciones en circulación). 

No disponemos en el dataframe del número de acciones que tiene en circulación cada compañía que forma el índice, y por tanto, **no nos es posible construir una reproducción fiel al original del índice S&P 500**. Aunque tuviésemos esta información, durante los 5 años de datos que tenemos han entrado y salido valores del índice, y no disponemos de los datos diarios de los que han salido. 

No obstante, podemos construir un índice sintético alterando las ponderaciones de los valores, utilizando el método *'EQUALLY WEIGHTED'*, en el que todos los valores tienen el mismo peso dentro del índice.

### CONSTRUCCIÓN DEL ÍNDICE
Para crear el indice necesitamos 2 cosas:
* *Un valor inicial* --> Tomaremos como referencia el valor del S&P el 10 de Agosto de 2012, un día hábil antes de la fecha a partir de la cual tenemos datos. Ese día el S&P 500 cerró a 1.405,87 y será el valor a partir del cual partamos.
* *Variaciones diarias del índice* --> A partir de las variaciones diarias de los 504 valores haremos que el índice se mueva diariamente.

Lo primero que tenemos que hacer entonces, es crear una fila para cada día con información del índice, que estará en varias columnas:
* *Date:* fecha en la que se recoge la información
* *Open:* precio en el que abre el índice. Carece de sentido real, ya que los precios a los que abre cada valor no tienen por qué coincidir en el tiempo (unos pueden cruzar operaciones nada más abrir el mercado, pero otros pueden hacerlo más tarde).
* *High:* precio más alto alcanzado por el índice. Carece también de sentido real, ya que los High de cada valor se producen en diferentes momentos durante la sesión.
* *Low:* precio más bajo alcanzado por el índice. Carece también de sentido real, ya que los Low de cada valor se producen en diferentes momentos durante la sesión.
* *Close:* precio al que cierra el índice. Es el dato más importante, y se construye a partir de los cierres de los 504 valores.

Pero antes de eso, debemos ordenar la tabla por valor y luego por fecha, para posteriormente añadir una columna a nuestro dataframe que recoja la variación diaria de cada valor:

In [95]:
sp['Change'] = np.zeros(sp.index.size)
for valor in lista_valores:

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
606771    0.0
606772    0.0
606773    0.0
606774    0.0
606775    0.0
606776    0.0
606777    0.0
606778    0.0
606779    0.0
606780    0.0
606781    0.0
606782    0.0
606783    0.0
606784    0.0
606785    0.0
606786    0.0
606787    0.0
606788    0.0
606789    0.0
606790    0.0
606791    0.0
606792    0.0
606793    0.0
606794    0.0
606795    0.0
606796    0.0
606797    0.0
606798    0.0
606799    0.0
606800    0.0
Name: Change, Length: 606801, dtype: float64

In [32]:
# CREACION DE DATAFRAME INDEXADO POR Name y LUEGO POR Date
byvalue = sp[0:1500].set_index(["Name", "Date"])
print(byvalue) 

                  Open   High    Low  Close      Volume
Name Date                                              
MMM  2012-08-13  92.29  92.59  91.74  92.40   2075391.0
     2012-08-14  92.36  92.50  92.01  92.30   1843476.0
     2012-08-15  92.00  92.74  91.94  92.54   1983395.0
     2012-08-16  92.75  93.87  92.21  93.74   3395145.0
     2012-08-17  93.93  94.30  93.59  94.24   3069513.0
     2012-08-20  94.00  94.17  93.55  93.89   1640008.0
     2012-08-21  93.98  94.10  92.99  93.21   2302988.0
     2012-08-22  92.56  93.36  92.43  92.68   2463908.0
     2012-08-23  92.65  92.68  91.79  91.98   1823757.0
     2012-08-24  92.03  92.97  91.94  92.83   1945796.0
     2012-08-27  92.92  92.97  92.40  92.59   1879969.0
     2012-08-28  92.36  92.77  92.20  92.30   1913066.0
     2012-08-29  92.50  92.80  92.30  92.43   1735933.0
     2012-08-30  92.05  92.15  91.30  91.76   1729576.0
     2012-08-31  92.44  93.00  91.90  92.60   1917265.0
     2012-09-04  92.03  92.33  91.10  91.68   25

In [107]:
lista_fechas = np.unique(sp['Date'])
print(sp[sp['Name']=='ZTS'])
#for valores in lista_valores:
    #for fechas in sp[sp['Name']==valores].index:
        #if fechas == fechassp['Change']=

              Date       Open       High    Low  Close      Volume Name  \
605660  2013-02-01  31.500000  31.740000  30.47  31.01  66789484.0  ZTS   
605661  2013-02-04  31.090000  31.990000  30.76  31.02   7695943.0  ZTS   
605662  2013-02-05  31.250000  31.980000  30.85  31.04   5014316.0  ZTS   
605663  2013-02-06  30.980000  31.430000  30.75  31.03   2126101.0  ZTS   
605664  2013-02-07  31.000000  32.730000  31.00  32.00   3800770.0  ZTS   
605665  2013-02-08  32.310000  33.480000  32.30  33.05   2599232.0  ZTS   
605666  2013-02-11  33.060000  33.500000  32.88  33.26   1487115.0  ZTS   
605667  2013-02-12  33.400000  34.000000  33.15  33.74   1637279.0  ZTS   
605668  2013-02-13  33.800000  33.800000  33.00  33.55   1269625.0  ZTS   
605669  2013-02-14  33.390000  33.450000  32.77  33.27   2954424.0  ZTS   
605670  2013-02-15  33.310000  34.090000  33.00  33.98   5415665.0  ZTS   
605671  2013-02-19  34.020000  34.070000  33.57  33.84   1206819.0  ZTS   
605672  2013-02-20  33.84

In [ ]:
# CREACIÓN DE UN DATAFRAME CON INFORMACIÓN DEL ÍNDICE


In [ ]:
ordenado = sp.sort_values(by=["Name","Date"])
ordenado['Change'] = np.zeros(ordenado.index.size)
ordenado = ordenado.round(2)
print(ordenado.head(5))
for i in range(len(ordenado)):
    if ordenado['Name'].iloc[i] == ordenado['Name'].iloc[i-1]:
        ordenado['Change'].iloc[i] = (ordenado['Close'].iloc[i] - ordenado['Change'].iloc[i-1]) / ordenado['Change'].iloc[i-1]

#PRUEBAS PARA DAR FORMATO %
#ordenado['Change'] = pd.Series(["{0:.2f}%"].format)
#ordenado['Change'] = pd.Series(["{0:.2f}%".format(val * 100) for val in ordenado['Change']], index = ordenado['Change'].index)
#ordenado['Change'].style.format([{'0:.2f}%'])
print(ordenado)
#ordenado.index = range(len(ordenado))
#ordenado['Change'] = np.zeros(ordenado.index.size)
#print (ordenado.iloc[0])
#print(ordenado)

In [81]:
ordenado['Name'].size

606801

In [88]:
ordenado.shape[0]

606801

In [ ]:
ordenado = sp.sort_values(by=["Name","Date"])
ordenado['Change'] = np.zeros(ordenado.index.size)
ordenado = ordenado.round(2)
print(ordenado.head(5))
for i in range(len(ordenado)):
    if ordenado['Name'].iloc[i] == ordenado['Name'].iloc[i-1]:
        ordenado['Change'].iloc[i] = (ordenado['Close'].iloc[i] - ordenado['Change'].iloc[i-1]) / ordenado['Change'].iloc[i-1]

A PARTIR DE AQUÍ SON PRUEBAS QUE HE IDO HACIENDO, PARA TESTEAR DISTINTAS FORMAS DE HACER LO MISMO.

In [32]:
columna_high=np.array(sp['High'])
print(columna_high[:2,0])
#columna_high.fillna()#Probar esto, a ver si funciona para rellenar los NaN. Crear las funciones.
# sp.fillna('Open':función sustituir Open, 'High': función sustituir High,'Low': función sustituir Low, /
#/'Volume': función sustituir Volume)
lista_valores= np.unique(sp['Name'])
ABT = np.array(sp[sp['Name']=="ABT"])
ABTbruto = sp[sp['Name']=="ABT"]
print(ABT)
print(ABTbruto)
print(len(sp[sp['Name']=="ABT"]['High'])-sp[sp['Name']=="ABT"]['High'].isnull().sum())
print(ABT[sp.index==2507,2])
if np.isnan(ABT['High'][2507])==False:  # Con esta sentencia evaluamos si el campo es NaN
    print(ABT[0,1])


IndexError: too many indices for array

In [ ]:
#Probar esto, a ver si funciona para rellenar los NaN. Crear las funciones.
# sp.fillna('Open':función sustituir Open, 'High': función sustituir High,'Low': función sustituir Low, /
#/'Volume': función sustituir Volume)
lista_valores= np.unique(sp['Name'])
ABT = np.array(sp[sp['Name']=="ABT"])
ABTbruto = sp[sp['Name']=="ABT"]
print(ABT)
print(ABTbruto)
print(len(sp[sp['Name']=="ABT"]['High'])-sp[sp['Name']=="ABT"]['High'].isnull().sum())
print(ABT[sp.index==2507,2])
if np.isnan(ABT['High'][2507])==False:  # Con esta sentencia evaluamos si el campo es NaN
    print(ABT[0,1])


In [72]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL HIGH
lista_valores = np.unique(sp['Name'])
col_high= sp['High']
dicc_high = {}

# En un diccionario vamos a almacenor, de cada valor, la media entre el High y el cierre. 
# Asignaremos a cada NaN que haya en la columna High el resultado de añadir este % al cierre de ese día para cada valor.

def calculo_reemplazo_nan_colhigh(lista_valores, dicc_high, col_high):
    for valor in lista_valores:
        acumulado = 0
        contador = 0
        posiciones_nan = []
        tabla_valor = (sp[sp['Name']==valor])
        for fila_valor in tabla_valor.index:
            if np.isnan(tabla_valor['High'][fila_valor])==False:
                    variacion = ( tabla_valor['High'][fila_valor] - tabla_valor['Close'][fila_valor] ) / tabla_valor['Close'][fila_valor]
                    acumulado = acumulado + variacion
                    contador = contador + 1
            else:
                posiciones_nan.append(fila_valor)
        dicc_high[valor] = (acumulado / contador)
        reemplazo = (acumulado / contador)
        col_high[posiciones]
        for posiciones in posiciones_nan:
            col_high[posiciones] = tabla_valor[posiciones, 'Close'] * (1+ reemplazo)
        
    return dicc_high, col_high

dicc_high = calculo_reemplazo_nan_colhigh(lista_valores, dicc_high, col_high)
print (col_high)

Int64Index([1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267,
            ...
            2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515],
           dtype='int64', length=1258)
             Date   Open   High    Low  Close      Volume Name
16272  2012-08-13  40.27  40.70  40.15  40.38   1860798.0    A
16273  2012-08-14  40.62  40.95  40.25  40.49   1994329.0    A
16274  2012-08-15  40.33  40.84  39.80  40.48   2838273.0    A
16275  2012-08-16  37.50  38.17  36.76  37.15  12797045.0    A
16276  2012-08-17  37.23  37.29  36.81  37.22   5377626.0    A
16277  2012-08-20  37.02  37.18  36.55  36.68   3972934.0    A
16278  2012-08-21  36.90  37.21  36.66  37.13   5375861.0    A
16279  2012-08-22  37.05  37.48  36.79  37.46   4383958.0    A
16280  2012-08-23  37.23  37.56  37.07  37.47   4492168.0    A
16281  2012-08-24  37.26  38.01  37.20  37.85   2877269.0    A
16282  2012-08-27  37.82  37.82  37.29  37.44   3058067.0    A
16283  2012-08-28  37.38  37.46  36.90  36.9

             Date    Open    High     Low   Close       Volume  Name
61817  2012-08-13   89.06   90.00   89.04   90.00   69707463.0  AAPL
61818  2012-08-14   90.27   91.23   90.03   90.24   85041824.0  AAPL
61819  2012-08-15   90.19   90.57   89.68   90.12   64377278.0  AAPL
61820  2012-08-16   90.17   90.97   90.07   90.91   63694204.0  AAPL
61821  2012-08-17   91.43   92.60   91.26   92.59  110689894.0  AAPL
61822  2012-08-20   92.86   95.02   92.84   95.02  153345689.0  AAPL
61823  2012-08-21   95.83   96.41   92.90   93.72  203176260.0  AAPL
61824  2012-08-22   93.49   95.57   92.59   95.55  141330637.0  AAPL
61825  2012-08-23   95.16   95.70   94.45   94.66  105029568.0  AAPL
61826  2012-08-24   94.22   95.64   93.65   94.75  109334113.0  AAPL
61827  2012-08-27   97.14   97.27   96.22   96.53  106728601.0  AAPL
61828  2012-08-28   96.43   96.59   95.81   96.40   66844323.0  AAPL
61829  2012-08-29   96.46   96.81   96.09   96.21   50701084.0  AAPL
61830  2012-08-30   95.81   95.94 

            Date    Open    High     Low   Close     Volume Name
3693  2012-08-13   61.27   61.55   60.94   61.50  1436939.0  ACN
3694  2012-08-14   61.55   61.63   61.05   61.26  2479628.0  ACN
3695  2012-08-15   61.03   61.46   60.68   60.82  1801610.0  ACN
3696  2012-08-16   61.08   61.37   60.56   61.15  1816897.0  ACN
3697  2012-08-17   61.63   61.98   61.02   61.08  2280824.0  ACN
3698  2012-08-20   61.12   61.28   60.68   60.79  2396342.0  ACN
3699  2012-08-21   61.02   61.36   60.74   60.98  1749846.0  ACN
3700  2012-08-22   60.71   61.96   60.55   61.55  3729681.0  ACN
3701  2012-08-23   61.40   61.49   60.64   60.83  2059817.0  ACN
3702  2012-08-24   60.56   61.64   60.41   61.53  2238286.0  ACN
3703  2012-08-27   61.55   61.79   61.11   61.25  1447189.0  ACN
3704  2012-08-28   60.73   61.15   60.53   60.85  1900722.0  ACN
3705  2012-08-29   60.99   61.26   60.53   61.02  1322050.0  ACN
3706  2012-08-30   60.80   61.37   60.50   61.15  2082668.0  ACN
3707  2012-08-31   61.64 

             Date    Open    High     Low   Close     Volume Name
28250  2012-08-13  131.15  132.95  130.84  132.72   485043.0  ADS
28251  2012-08-14  133.00  134.63  133.00  134.57   623984.0  ADS
28252  2012-08-15  133.71  134.85  133.25  134.39   234515.0  ADS
28253  2012-08-16  134.00  136.39  133.91  136.35   629797.0  ADS
28254  2012-08-17  136.56  137.83  135.88  137.47   741059.0  ADS
28255  2012-08-20  137.88  138.33  134.91  136.00   387880.0  ADS
28256  2012-08-21  136.00  136.09  134.75  136.04   357716.0  ADS
28257  2012-08-22  136.06  137.37  135.33  136.86   387488.0  ADS
28258  2012-08-23  136.43  136.85  135.70  136.55   268240.0  ADS
28259  2012-08-24  136.69  138.68  136.18  138.49   407758.0  ADS
28260  2012-08-27  138.73  139.72  138.07  138.99   797749.0  ADS
28261  2012-08-28  138.52  138.98  137.09  138.08   671478.0  ADS
28262  2012-08-29  138.24  138.67  137.41  138.43   383660.0  ADS
28263  2012-08-30  137.87  137.87  136.10  137.17   376202.0  ADS
28264  201

             Date    Open    High     Low   Close      Volume Name
12498  2012-08-13   37.59   37.70   37.29   37.48   1982302.0  AET
12499  2012-08-14   37.55   37.69   36.88   36.97   2209796.0  AET
12500  2012-08-15   37.06   37.84   37.00   37.65   2155442.0  AET
12501  2012-08-16   37.56   38.17   37.42   38.17   2902286.0  AET
12502  2012-08-17   38.24   38.29   37.77   38.04   2253655.0  AET
12503  2012-08-20   39.70   40.24   39.00   40.18  15124868.0  AET
12504  2012-08-21   39.87   40.47   39.71   39.90   6390164.0  AET
12505  2012-08-22   39.82   39.87   39.40   39.57   4052159.0  AET
12506  2012-08-23   39.55   39.72   39.25   39.30   3199762.0  AET
12507  2012-08-24   39.23   39.39   38.66   38.72   4418183.0  AET
12508  2012-08-27   38.74   39.19   38.62   39.00   2875131.0  AET
12509  2012-08-28   38.81   39.03   38.51   38.53   2574012.0  AET
12510  2012-08-29   38.66   39.36   38.65   39.12   2935843.0  AET
12511  2012-08-30   38.86   39.06   38.73   38.80   3197112.0 

...

              Date   Open   High    Low  Close      Volume Name
578438  2012-08-13  35.00  35.05  34.88  35.01   1463755.0   WM
578439  2012-08-14  35.14  35.22  35.00  35.21   1898566.0   WM
578440  2012-08-15  35.12  35.41  35.06  35.33   2020392.0   WM
578441  2012-08-16  35.28  35.70  35.15  35.68   1997483.0   WM
578442  2012-08-17  35.65  35.66  35.43  35.66   2009147.0   WM
578443  2012-08-20  34.93  34.99  33.88  34.60   7905791.0   WM
578444  2012-08-21  34.50  34.80  34.44  34.48   4085529.0   WM
578445  2012-08-22  34.39  34.60  34.31  34.40   2507038.0   WM
578446  2012-08-23  34.37  34.48  34.25  34.34   2939426.0   WM
578447  2012-08-24  34.22  34.54  34.16  34.28   2307788.0   WM
578448  2012-08-27  34.29  34.40  34.02  34.09   2395868.0   WM
578449  2012-08-28  34.05  34.38  34.01  34.23   2342121.0   WM
578450  2012-08-29  34.19  34.76  34.19  34.65   3672835.0   WM
578451  2012-08-30  34.45  34.65  34.35  34.53   2360226.0   WM
578452  2012-08-31  34.61  34.80  34.46 

              Date   Open   High    Low  Close     Volume Name
585986  2012-08-13  17.48  17.55  17.25  17.52  3204215.0   WU
585987  2012-08-14  17.57  17.78  17.53  17.63  4714474.0   WU
585988  2012-08-15  17.60  17.72  17.56  17.59  3462991.0   WU
585989  2012-08-16  17.64  17.80  17.57  17.73  2638691.0   WU
585990  2012-08-17  17.81  17.82  17.62  17.69  4206325.0   WU
585991  2012-08-20  17.67  17.93  17.63  17.82  3832496.0   WU
585992  2012-08-21  17.84  17.93  17.68  17.75  3054945.0   WU
585993  2012-08-22  17.70  17.72  17.35  17.48  4564356.0   WU
585994  2012-08-23  17.47  17.51  17.31  17.36  2332190.0   WU
585995  2012-08-24  17.35  17.45  17.26  17.42  2324898.0   WU
585996  2012-08-27  17.45  17.56  17.37  17.38  2586598.0   WU
585997  2012-08-28  17.36  17.48  17.29  17.33  3526341.0   WU
585998  2012-08-29  17.35  17.57  17.29  17.53  3260095.0   WU
585999  2012-08-30  17.40  17.48  17.32  17.41  3096296.0   WU
586000  2012-08-31  17.48  17.67  17.34  17.61  4540408

              Date   Open   High    Low  Close     Volume Name
596588  2012-08-13  28.81  28.96  28.70  28.75  1393602.0  XEL
596589  2012-08-14  28.77  28.96  28.72  28.77  1493137.0  XEL
596590  2012-08-15  28.77  28.80  28.62  28.62  1213183.0  XEL
596591  2012-08-16  28.60  28.78  28.48  28.57  1421841.0  XEL
596592  2012-08-17  28.65  28.67  28.48  28.49  1613656.0  XEL
596593  2012-08-20  28.50  28.57  28.35  28.48  1231383.0  XEL
596594  2012-08-21  28.46  28.55  28.25  28.27  1433306.0  XEL
596595  2012-08-22  28.21  28.34  28.14  28.18  1394933.0  XEL
596596  2012-08-23  28.18  28.25  27.64  27.65  3389361.0  XEL
596597  2012-08-24  27.65  27.96  27.63  27.93  2351395.0  XEL
596598  2012-08-27  28.00  28.10  27.92  27.97  1569017.0  XEL
596599  2012-08-28  27.92  28.03  27.79  27.95  1265017.0  XEL
596600  2012-08-29  27.97  28.02  27.84  27.95  1364481.0  XEL
596601  2012-08-30  27.84  27.92  27.71  27.84  1205516.0  XEL
596602  2012-08-31  27.96  27.96  27.74  27.89  1952785

              Date   Open   High    Low  Close     Volume Name
597846  2012-08-13  28.60  28.80  28.60  28.76  2264972.0  XRX
597847  2012-08-14  28.88  28.96  28.56  28.72  1920328.0  XRX
597848  2012-08-15  28.72  29.00  28.64  28.96  1982004.0  XRX
597849  2012-08-16  28.96  29.44  28.84  29.36  2041150.0  XRX
597850  2012-08-17  29.36  29.68  29.20  29.64  2330570.0  XRX
597851  2012-08-20  29.52  29.90  29.16  29.68  2283682.0  XRX
597852  2012-08-21  29.64  30.08  29.54  29.68  1942464.0  XRX
597853  2012-08-22  29.64  29.64  29.08  29.16  2015266.0  XRX
597854  2012-08-23  29.00  29.36  28.72  29.20  2084998.0  XRX
597855  2012-08-24  29.20  29.76  28.92  29.52  1779596.0  XRX
597856  2012-08-27  29.52  29.60  29.04  29.20  1427164.0  XRX
597857  2012-08-28  29.04  29.68  29.00  29.52  1780065.0  XRX
597858  2012-08-29  29.60  29.72  29.28  29.56  1519128.0  XRX
597859  2012-08-30  29.32  29.60  29.20  29.32  1540882.0  XRX
597860  2012-08-31  29.60  29.80  29.32  29.48  2424052

              Date   Open   High    Low  Close      Volume Name
605660  2013-02-01  31.50  31.74  30.47  31.01  66789484.0  ZTS
605661  2013-02-04  31.09  31.99  30.76  31.02   7695943.0  ZTS
605662  2013-02-05  31.25  31.98  30.85  31.04   5014316.0  ZTS
605663  2013-02-06  30.98  31.43  30.75  31.03   2126101.0  ZTS
605664  2013-02-07  31.00  32.73  31.00  32.00   3800770.0  ZTS
605665  2013-02-08  32.31  33.48  32.30  33.05   2599232.0  ZTS
605666  2013-02-11  33.06  33.50  32.88  33.26   1487115.0  ZTS
605667  2013-02-12  33.40  34.00  33.15  33.74   1637279.0  ZTS
605668  2013-02-13  33.80  33.80  33.00  33.55   1269625.0  ZTS
605669  2013-02-14  33.39  33.45  32.77  33.27   2954424.0  ZTS
605670  2013-02-15  33.31  34.09  33.00  33.98   5415665.0  ZTS
605671  2013-02-19  34.02  34.07  33.57  33.84   1206819.0  ZTS
605672  2013-02-20  33.84  33.88  32.60  32.72   1843361.0  ZTS
605673  2013-02-21  32.50  32.80  31.86  32.56   3242213.0  ZTS
605674  2013-02-22  32.78  32.93  32.50 

In [110]:
tabla1 = (sp[sp['Name'] == "ABT"])
tabla2 = np.array(sp[sp['Name'] == "ABT"])
print (tabla1)
print (tabla2)

            Date   Open   High    Low  Close      Volume Name
1258  2012-08-13  32.93  33.02  32.84  33.00   5290056.0  ABT
1259  2012-08-14  33.08  33.14  32.95  33.08   5499012.0  ABT
1260  2012-08-15  33.12  33.34  33.04  33.21   7044866.0  ABT
1261  2012-08-16  33.16  33.25  33.00  33.22   5353566.0  ABT
1262  2012-08-17  33.30  33.30  32.84  32.96   8122218.0  ABT
1263  2012-08-20  32.99  33.05  32.84  32.86   6692920.0  ABT
1264  2012-08-21  32.86  32.98  32.62  32.65   8420758.0  ABT
1265  2012-08-22  32.56  32.86  32.54  32.77   6312514.0  ABT
1266  2012-08-23  32.80  32.89  32.57  32.66   7002542.0  ABT
1267  2012-08-24  32.54  32.95  32.54  32.92   7649672.0  ABT
1268  2012-08-27  32.90  32.99  32.78  32.86   5946134.0  ABT
1269  2012-08-28  32.78  32.95  32.53  32.86   5506126.0  ABT
1270  2012-08-29  32.91  33.00  32.80  32.80   5428042.0  ABT
1271  2012-08-30  32.68  32.81  32.58  32.70   6047650.0  ABT
1272  2012-08-31  32.75  32.95  32.70  32.77   8269468.0  ABT
1273  20

In [134]:
# INTENTO SUSITUTIR NAN CON UN BUCLE, pero da el mismo error, al intentar acceder al valor del diccinario se queda pillado.
macrotabla = np.array(sp)
print(macrotabla[2506])
for i in range (len(macrotabla)):
    if np.isnan(macrotabla[i,2]) == True:
        print(macrotabla[i,6])
        #macrotabla[i,2] = dicc_high[macrotabla[i,6]]
print(macrotabla[2506]) # --> Sigue teniendo los NaN

['2017-07-31' nan nan 49.16 49.18 6036176.0 'ABT']


KeyError: 'ABT'

In [104]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL HIGH
lista_valores = np.unique(sp['Name'])
col_high= sp['High']
dicc_high = {}
posiciones_nan= np.where(np.isnan(col_high))
# En un diccionario vamos a almacenor, de cada valor, la media entre el High y el cierre. 
# Asignaremos a cada NaN que haya en la columna High el resultado de añadir este % al cierre de ese día para cada valor.

def calculo_reemplazo_nan_colhigh(lista_valores, dicc_high, col_high):
    for valor in lista_valores:
        acumulado = 0
        contador = 0
        posiciones_nan = []
        tabla_valor = (sp[sp['Name']==valor])
        for fila_valor in tabla_valor.index:
            if np.isnan(tabla_valor['High'][fila_valor])==False:
                    variacion = ( tabla_valor['High'][fila_valor] - tabla_valor['Close'][fila_valor] ) / tabla_valor['Close'][fila_valor]
                    acumulado = acumulado + variacion
                    contador = contador + 1
            else:
                posiciones_nan.append(fila_valor)
        dicc_high[valor] = (acumulado / contador)
        reemplazo = (acumulado / contador)
        #col_high[posiciones]
        for posiciones in posiciones_nan:
            col_high[posiciones] = tabla_valor[posiciones, 'Close'] * (1+ reemplazo)
        
    return dicc_high, col_high

dicc_high = calculo_reemplazo_nan_colhigh(lista_valores, dicc_high, col_high)
print (col_high)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [3]:
print(sp['High'].index)

RangeIndex(start=0, stop=606801, step=1)


In [67]:
col_avg_high = []
for i in range(len(col_high)):
    col_avg_high.append(dicc_high[col_name[i]])
print(col_avg_high)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [38]:
acciones = []
reemplazo_high = []
for valores in dicc_high:
    acciones.append(valores)
    reemplazo_high.append(dicc_high[valores])
print(acciones, reemplazo_high)
print(acciones.index("BAX"))

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AET', 'AFL', 'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMD', 'AME', 'AMG', 'AMGN', 'AMP', 'AMZN', 'ANDV', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APC', 'APD', 'APH', 'ARE', 'ARNC', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AYI', 'AZO', 'BA', 'BAC', 'BAX', 'BBT', 'BBY', 'BCR', 'BDX', 'BEN', 'BF.B', 'BHF', 'BHGE', 'BIIB', 'BK', 'BLK', 'BLL', 'BMY', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CA', 'CAG', 'CAH', 'CAT', 'CB', 'CBG', 'CBOE', 'CBS', 'CCI', 'CCL', 'CELG', 'CERN', 'CF', 'CFG', 'CHD', 'CHK', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COH', 'COL', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CRM', 'CSCO', 'CSRA', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DI

In [83]:
# INTENTO DE REEMPLAZAR COLUMNAS NAN TARBAJANDO CON ARRAYS DE CADA COLUMNA EN LUGAR DE TABLA
posiciones_nan_High= np.where(np.isnan(col_high))
#print(posiciones_nan_High)
for posicion in posiciones_nan_High:
    print(col_close[posicion] * (1+(dicc_high[col_name[posicion]])))

def reemplazo_nan_colhigh(posiciones, high, close, name, dicc):
    for posicion in posiciones_nan_High:
        high[posicion] = close[posicion] * (1+(dicc[name[posicion]]))
    return col_high

col_high = reemplazo_nan_colhigh(posiciones_nan_High, col_high, col_close, col_name, dicc_high)
print(col_high)
#col_high.fillna(funcion)

[  49.18   69.91  202.65  154.31   59.79  142.15   85.23  115.8    81.24
  252.33   40.53   91.     64.97   56.1    93.82   61.58   99.53  186.21
  138.17   53.55   49.48   24.79   58.79  105.27  192.35   36.89   24.12
  320.6    60.48  242.46   46.74  120.91   26.62   69.08   49.4    24.87
   77.26   66.25   66.78  113.95   65.83   79.42    4.96  343.77  173.56
   47.14   45.84   72.2   193.35  243.87   29.14  100.58   32.61  167.9
   79.93   35.69   83.88   64.78  128.28   90.42   49.36   75.16   84.
   64.24   28.59   82.21   78.38   83.16   78.25  131.67  115.18   59.61
   63.7   145.44  261.7    98.99  132.63  208.03   91.22   31.91   41.13
   43.43   76.38   47.     50.     14.62  196.33   22.61   35.98   94.37
   42.44   22.92   48.67  114.47   55.     80.34   50.62  105.31   17.51
   62.53   34.17   19.1   231.2   140.71   87.88  133.18  121.9    38.95
   27.95   20.18   41.35  174.97  158.91   48.18   52.44   26.03   82.66
   73.06   57.39   23.75   77.97   38.13  155.14  161.

TypeError: unhashable type: 'numpy.ndarray'

In [65]:
col_high = sp['High']
col_name = sp['Name']
col_close = sp['Close']
posiciones_nan_high= np.where(np.isnan(col_high))
print(posiciones_nan_high)
#print(col_high[2506])
print(reemplazo_high[acciones.index(col_name[606791])])
def reemplazo_nan_colhigh(col_high, col_name, col_close, posiciones_nan_high, acciones, reemplazo_high):
    for posicion in posiciones_nan_high:
        reemplazo = reemplazo_high[acciones.index(col_name[posicion])]
        col_high[posicion] = reemplazo
        
        #col_high[posicion] = col_close[posicion] #* (1 + dicc_high[sp['Name'][posicion]])
    return col_high

col_high2 = reemplazo_nan_colhigh(col_high, col_name, col_close, posiciones_nan_high, acciones, reemplazo_high)
print(col_high2)

(array([  2506,   3683,   7457,  13746,  17520,  18778,  21294,  22552,
        27266,  28240,  30756,  32014,  35788,  38304,  46778,  48036,
        54326,  56842,  58033,  59291,  60549,  66838,  68096,  69354,
        75644,  76930,  79446,  81962,  83220,  93284,  94542,  95800,
        97058,  97075,  99981, 103755, 107529, 110045, 111302, 112560,
       115076, 117592, 125452, 127968, 131742, 145049, 146307, 148823,
       157629, 158887, 160145, 163122, 163560, 166075, 167333, 168591,
       171107, 172365, 173623, 174881, 176139, 186203, 189977, 191235,
       195009, 196267, 197627, 200143, 201324, 203840, 206356, 208872,
       212820, 214078, 217237, 218495, 233591, 234849, 236107, 238623,
       242397, 243655, 244913, 247442, 247444, 251497, 256529, 259045,
       261561, 265335, 270359, 271617, 272875, 274133, 275391, 277907,
       280423, 282939, 284655, 285578, 290610, 293126, 294384, 298952,
       301468, 309965, 318159, 320106, 322622, 328912, 332686, 336992,
     

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
#OBTENER PROMEDIO POR VALOR ENTRE EL CIERRE Y EL HIGH
lista_valores= np.unique(sp['Name'])
list_high=np.array([])

# En un diccionario almaceno de cada valor, la media entre el High y el cierre. Este valor lo tomarán los NaN en la col High.
for valor in lista_valores:
    print(valor)
    acumulado = 0
    contador = 0
    for fila_valor in sp[sp['Name'] == valor].index:
        print(fila_valor)
        if np.isnan(sp['High'][fila_valor])==False:
            variacion = ( sp['High'][fila_valor] - sp['Close'][fila_valor] ) / sp['Close'][fila_valor]
            acumulado = acumulado + variacion
            contador = contador + 1
    #list_high[fila_valor].append(acumulado / contador)  

print(list_high)

#def reemplazo_nan_high()

In [202]:
#print(dicc_high["ABT"])
highcol = sp['High']
namecol = sp['Name']
posiciones_nan_High= np.where(np.isnan(highcol))
print(posiciones_nan_High)
for clave in dicc_high:
    print(dicc_high[clave])

for i in posiciones_nan_High:
    rick=namecol[i]
    print(rick)
    print(dicc_high[rick])
#    print(dicc_high[i])


(array([  2506,   3683,   7457,  13746,  17520,  18778,  21294,  22552,
        27266,  28240,  30756,  32014,  35788,  38304,  46778,  48036,
        54326,  56842,  58033,  59291,  60549,  66838,  68096,  69354,
        75644,  76930,  79446,  81962,  83220,  93284,  94542,  95800,
        97058,  97075,  99981, 103755, 107529, 110045, 111302, 112560,
       115076, 117592, 125452, 127968, 131742, 145049, 146307, 148823,
       157629, 158887, 160145, 163122, 163560, 166075, 167333, 168591,
       171107, 172365, 173623, 174881, 176139, 186203, 189977, 191235,
       195009, 196267, 197627, 200143, 201324, 203840, 206356, 208872,
       212820, 214078, 217237, 218495, 233591, 234849, 236107, 238623,
       242397, 243655, 244913, 247442, 247444, 251497, 256529, 259045,
       261561, 265335, 270359, 271617, 272875, 274133, 275391, 277907,
       280423, 282939, 284655, 285578, 290610, 293126, 294384, 298952,
       301468, 309965, 318159, 320106, 322622, 328912, 332686, 336992,
     

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [196]:
highcol = sp['High']
posiciones_nan_High= np.where(np.isnan(highcol))

print(posiciones_nan_High)

for i in posiciones_nan_High:
    print(sp.iloc[i])
    rick = sp.iloc[i]['Name']
    #print(rick)
    highcol[i] = (dicc_high[rick]) #sp['Name'][i] #* 
#print(highcol)

(array([  2506,   3683,   7457,  13746,  17520,  18778,  21294,  22552,
        27266,  28240,  30756,  32014,  35788,  38304,  46778,  48036,
        54326,  56842,  58033,  59291,  60549,  66838,  68096,  69354,
        75644,  76930,  79446,  81962,  83220,  93284,  94542,  95800,
        97058,  97075,  99981, 103755, 107529, 110045, 111302, 112560,
       115076, 117592, 125452, 127968, 131742, 145049, 146307, 148823,
       157629, 158887, 160145, 163122, 163560, 166075, 167333, 168591,
       171107, 172365, 173623, 174881, 176139, 186203, 189977, 191235,
       195009, 196267, 197627, 200143, 201324, 203840, 206356, 208872,
       212820, 214078, 217237, 218495, 233591, 234849, 236107, 238623,
       242397, 243655, 244913, 247442, 247444, 251497, 256529, 259045,
       261561, 265335, 270359, 271617, 272875, 274133, 275391, 277907,
       280423, 282939, 284655, 285578, 290610, 293126, 294384, 298952,
       301468, 309965, 318159, 320106, 322622, 328912, 332686, 336992,
     

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [122]:
ABT = sp[sp['Name']=="ABT"]
acumulado = 0
contador = 0
for fila_valor in sp[sp['Name'] == "ABT"].index:
    if np.isnan(sp['High'][fila_valor])==False:
        variacion = ( sp['High'][fila_valor] - sp['Close'][fila_valor] ) / sp['Close'][fila_valor]
        acumulado = acumulado + variacion
        contador = contador + 1
    #campos_buenos= len(sp[sp['Name'] == valor]['High']) - sp[sp['Name']==valor]['High'].isnull().sum()
promedio= (acumulado / contador)
print ( promedio)

0.00751108215416


In [60]:
# COLUMNA HIGH --> RELLENAR LOS NaN
# Los High los podemos rellenar con la media de la diferencia diaria que cada valor tenga desde el precio de cierre diario
# hasta el High diario.
highcol = sp['High']
posiciones_nan_High= np.where(np.isnan(highcol))

def highfun (highcol):
    posiciones_nan_High= np.where(np.isnan(highcol))
    for filas in posiciones_nan_High:
        

sp['High'] = highfun(sp['High'])

In [56]:
# COLUMNA HIGH --> RELLENAR LOS NaN
# Los High los podemos rellenar con la media de la diferencia diaria que cada valor tenga desde el precio de cierre diario
# hasta el High diario.
col_high
posiciones_nan_High= np.where(np.isnan(col_high))
print(posiciones_nan_High)

def highfun (highcol):
    #posiciones_nan_High= np.where(np.isnan(highcol))
    for filas in posiciones_nan_High:
        col

col_high = highfun()

(array([  2506,   3683,   7457,  13746,  17520,  18778,  21294,  22552,
        27266,  28240,  30756,  32014,  35788,  38304,  46778,  48036,
        54326,  56842,  58033,  59291,  60549,  66838,  68096,  69354,
        75644,  76930,  79446,  81962,  83220,  93284,  94542,  95800,
        97058,  97075,  99981, 103755, 107529, 110045, 111302, 112560,
       115076, 117592, 125452, 127968, 131742, 145049, 146307, 148823,
       157629, 158887, 160145, 163122, 163560, 166075, 167333, 168591,
       171107, 172365, 173623, 174881, 176139, 186203, 189977, 191235,
       195009, 196267, 197627, 200143, 201324, 203840, 206356, 208872,
       212820, 214078, 217237, 218495, 233591, 234849, 236107, 238623,
       242397, 243655, 244913, 247442, 247444, 251497, 256529, 259045,
       261561, 265335, 270359, 271617, 272875, 274133, 275391, 277907,
       280423, 282939, 284655, 285578, 290610, 293126, 294384, 298952,
       301468, 309965, 318159, 320106, 322622, 328912, 332686, 336992,
     

In [74]:
col_high
posiciones_nan_High= np.where(np.isnan(col_high))
print(posiciones_nan_High)
print(col_close[2506] * (1+(dicc_high[col_name[2506]])))

(array([  2506,   3683,   7457,  13746,  17520,  18778,  21294,  22552,
        27266,  28240,  30756,  32014,  35788,  38304,  46778,  48036,
        54326,  56842,  58033,  59291,  60549,  66838,  68096,  69354,
        75644,  76930,  79446,  81962,  83220,  93284,  94542,  95800,
        97058,  97075,  99981, 103755, 107529, 110045, 111302, 112560,
       115076, 117592, 125452, 127968, 131742, 145049, 146307, 148823,
       157629, 158887, 160145, 163122, 163560, 166075, 167333, 168591,
       171107, 172365, 173623, 174881, 176139, 186203, 189977, 191235,
       195009, 196267, 197627, 200143, 201324, 203840, 206356, 208872,
       212820, 214078, 217237, 218495, 233591, 234849, 236107, 238623,
       242397, 243655, 244913, 247442, 247444, 251497, 256529, 259045,
       261561, 265335, 270359, 271617, 272875, 274133, 275391, 277907,
       280423, 282939, 284655, 285578, 290610, 293126, 294384, 298952,
       301468, 309965, 318159, 320106, 322622, 328912, 332686, 336992,
     

In [58]:
# Los vacíos de la columna Open se pueden rellenar con un aleatorio que esté en el rango entre el máx y mín del día
def openfun():
    opencol=sp['Open']
    for i in range(len(opencol)):
        if opencol[i].isnan():
            opencol[i]=randint(sp['Low'][i], sp['High'][i])
    
# sp['Open'].fillna(función sustituir Open)



In [25]:
opencol=sp['Open']
opencol=opencol.fillna(0) # o cambiar esto por opencol=opencol.fillna(funcion)
for valores in opencol:
    print(valores)
    if sp['Open'][valores]==0:
        opencol[valores]=1

print(opencol[1])
#print(opencol)

606801
92.29


TypeError: cannot do label indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [92.29] of <class 'numpy.float64'>

In [22]:
for i in range(sp.index):
    if sp['Open'[i]]
sp.iloc[3:4,:]

,Date,Open,High,Low,Close,Volume,Name
3,2012-08-16,92.75,93.87,92.21,93.74,3395145.0,MMM


In [23]:
sp.loc[3:4,:]

,Date,Open,High,Low,Close,Volume,Name
3,2012-08-16,92.75,93.87,92.21,93.74,3395145.0,MMM
4,2012-08-17,93.93,94.30,93.59,94.24,3069513.0,MMM


In [27]:
sp.loc[3:4,'High']

3    93.87
4    94.30
Name: High, dtype: float64

In [28]:
sp.iloc[3:4,'High']

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types